In [ ]:
pip install qiskit[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram, plot_state_qsphere
from qiskit.quantum_info import Statevector, random_statevector

In [ ]:
oracle_type = input('Would you like to test using a constant or balanced oracle? ')
oracle_type = oracle_type.lower()
oracle_size = int(input('How many input arguments should this oracle function take in? '))

Would you like to test using a constant or balanced oracle? Balanced
How many input arguments should this oracle function take in? 2


In [ ]:
def measure_and_report(qc, oracle_size):
  for i in range(oracle_size+1):
    print(f'measuring qubit {i} in classical bit 0')
    qc.measure([0, i], [0, i])
    
  simulator = QasmSimulator()

  job = simulator.run(qc)
  result = job.result()
  counts = result.get_counts(qc)

  return counts

In [ ]:
def simulate_qc(qc):
  aer_sim = Aer.get_backend('aer_simulator')
  results = aer_sim.run(qc).result()
  final_result = results.get_counts()

  plot_histogram(final_result)

In [ ]:
def apply_hadamard_gates(qc, oracle_size):
  for i in range(oracle_size+1):
    qc.h(i)
  return qc

In [ ]:
def create_constant_oracle(oracle_size):
  pass

In [ ]:
def create_balanced_oracle(oracle_size):
  oracle = QuantumCircuit(oracle_size+1)
  oracle.barrier()

  oracle.z(0)
  oracle.cz(1, oracle_size)

  oracle.barrier()

  return oracle
  # oracle = QuantumCircuit(oracle_size+1)
  # oracle.barrier()

  # for i in range(oracle_size):
  #   oracle.cx(i, oracle_size)
  
  # oracle.barrier()
  
  # return oracle

In [ ]:
test = Statevector.from_label('000')

main_circuit = QuantumCircuit(oracle_size + 1)
main_circuit.x(oracle_size)
print(main_circuit)
test.evolve(main_circuit)

          
q_0: ─────
          
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
Statevector([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))


In [ ]:
main_circuit = apply_hadamard_gates(main_circuit, 2)
print(main_circuit)

     ┌───┐     
q_0: ┤ H ├─────
     ├───┤     
q_1: ┤ H ├─────
     ├───┤┌───┐
q_2: ┤ X ├┤ H ├
     └───┘└───┘


In [ ]:
if oracle_type == 'balanced':
  oracle = create_balanced_oracle(oracle_size)
  main_circuit = main_circuit.compose(oracle)
  print(main_circuit)
  
  main_circuit = apply_hadamard_gates(main_circuit, oracle_size)
  print(main_circuit)

  result = Statevector.from_label('000')
  print(result.evolve(main_circuit))

  # simulate_qc(main_circuit)

elif oracle_type == 'constant':
  oracle = create_constant_oracle(oracle_size)

  main_circuit = apply_hadamard_gates(main_circuit, oracle_size)
  print(main_circuit)

  result = Statevector.from_label('000')
  print(result.evolve(main_circuit))

     ┌───┐      ░ ┌───┐ ░ 
q_0: ┤ H ├──────░─┤ Z ├─░─
     ├───┤      ░ └───┘ ░ 
q_1: ┤ H ├──────░───■───░─
     ├───┤┌───┐ ░   │   ░ 
q_2: ┤ X ├┤ H ├─░───■───░─
     └───┘└───┘ ░       ░ 
     ┌───┐      ░ ┌───┐ ░ ┌───┐
q_0: ┤ H ├──────░─┤ Z ├─░─┤ H ├
     ├───┤      ░ └───┘ ░ ├───┤
q_1: ┤ H ├──────░───■───░─┤ H ├
     ├───┤┌───┐ ░   │   ░ ├───┤
q_2: ┤ X ├┤ H ├─░───■───░─┤ H ├
     └───┘└───┘ ░       ░ └───┘
Statevector([ 3.11164266e-19+0.j,  5.00000000e-01+0.j, -3.11164266e-19+0.j,
             -5.00000000e-01+0.j,  3.11164266e-19+0.j,  5.00000000e-01+0.j,
              3.11164266e-19+0.j,  5.00000000e-01+0.j],
            dims=(2, 2, 2))


QiskitError: ignored